In [ ]:
#Importing library
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np
from glob import glob

In [ ]:
np.random.seed(1000)
#Instantiation
AlexNet = Sequential()
# useful for getting number of output classes
from google.colab import drive
drive.mount("/content/drive/")
folders = glob('/content/drive/My Drive/db/train/*')

Mounted at /content/drive/


In [ ]:

#1st Convolutional Layer
AlexNet.add(Conv2D(filters=96, input_shape=(150,150,3), kernel_size=(11,11), strides=(4,4), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#2nd Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#3rd Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#4th Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#5th Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#Passing it to a Fully Connected layer
AlexNet.add(Flatten())
# 1st Fully Connected Layer
AlexNet.add(Dense(4096, input_shape=(150,150,3,)))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
# Add Dropout to prevent overfitting
AlexNet.add(Dropout(0.4))

#2nd Fully Connected Layer
AlexNet.add(Dense(4096))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#3rd Fully Connected Layer
AlexNet.add(Dense(1000))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#Output Layer
AlexNet.add(Dense(10))
AlexNet.add(BatchNormalization())
AlexNet.add(Dense(len(folders), activation='softmax'))

#Model Summary
AlexNet.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 38, 38, 96)        34944     
_________________________________________________________________
batch_normalization (BatchNo (None, 38, 38, 96)        384       
_________________________________________________________________
activation (Activation)      (None, 38, 38, 96)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 19, 19, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 19, 19, 256)       614656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 19, 19, 256)       1024      
_________________________________________________________________
activation_1 (Activation)    (None, 19, 19, 256)       0

In [ ]:
# re-size all the images to this
IMAGE_SIZE = [150, 150]

train_path = '/content/drive/My Drive/db/train'
valid_path = '/content/drive/My Drive/db/test'

In [ ]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/My Drive/db/train',
                                                 target_size = (150, 150),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/drive/My Drive/db/test',
                                            target_size = (150, 150),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 2950 images belonging to 3 classes.
Found 690 images belonging to 3 classes.


In [ ]:
AlexNet.compile(loss = keras.losses.categorical_crossentropy, optimizer= 'adam', metrics=['accuracy'])
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint_filepath = 'saved_model/'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
# fit the model
# Run the cell. It will take some time to execute
alexnet_model = AlexNet.fit(
  training_set,
  validation_data=test_set,
  epochs=15,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set),
  callbacks=[model_checkpoint_callback]
)

Epoch 1/15
93/93 [==============================] - ETA: 0s - loss: 0.5268 - accuracy: 0.8536 WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/tracking/tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: saved_model/assets
93/93 [==============================] - 1589s 17s/step - loss: 0.5268 - accuracy: 0.8536 - val_loss: 1.8737 - val_accuracy: 0.2522
Epoch 2/15
93/93 [==============================] - 333s 4s/step - loss: 0.3635 - accuracy: 0.8976 - val_loss: 2.2978 - val_accuracy: 0.4507
Epoch 3/15
93/93 [==============================] - 333s 4s/step - loss: 0.3040 - accuracy: 0.9051 - val_lo

In [ ]:
AlexNet.save('saved_model/alexnet.h5')
from google.colab import files
files.download("saved_model/alexnet.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/db/models/alexnet.h5')
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_60 (Conv2D)           (None, 38, 38, 96)        34944     
_________________________________________________________________
batch_normalization_99 (Batc (None, 38, 38, 96)        384       
_________________________________________________________________
activation_98 (Activation)   (None, 38, 38, 96)        0         
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 19, 19, 96)        0         
_________________________________________________________________
conv2d_61 (Conv2D)           (None, 19, 19, 256)       614656    
_________________________________________________________________
batch_normalization_100 (Bat (None, 19, 19, 256)       1024      
_________________________________________________________________
activation_99 (Activation)   (None, 19, 19, 256)     